In [122]:
#reading data
import pandas as pd
patent_data = pd.read_csv("2023.csv")


C:\Users\rafbl\AppData\Local\Temp\ipykernel_8648\4055054621.py:3: DtypeWarning: Columns (0,18,21,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  patent_data = pd.read_csv("2023.csv")


In [123]:
# preparing data for analysis by keeping and creating relevant columns and removing Null values 
patent_data["assignee_type"] = patent_data["assignee_ind"].map({1:"Individual", 0:"Organization"})
patent_data["years_to_grant"] = patent_data["grant_year"] - patent_data["application_year"]
patent_data = patent_data.loc[patent_data["first_wipo_field_title"].notna()]
patent_data = patent_data.loc[:,["years_to_grant","first_wipo_field_title","assignee_type"]]
patent_data.rename(columns={"first_wipo_field_title":"field"},inplace=True)
patent_data


,years_to_grant,field,assignee_type
0,4,Other special machines,Organization
1,3,Other special machines,Organization
2,3,Other special machines,Organization
3,3,Other special machines,Organization
4,6,Other special machines,Organization
...,...,...,...
328068,2,Computer technology,Organization
328071,2,Computer technology,Organization
328072,3,Computer technology,Organization
328074,2,Computer technology,Organization


In [124]:
# Setting all the fields here we will keep the top fields (>7000 patents ) and set the rest to other

top_fields = [
    "Computer technology", "Digital communication", "Electrical machinery, apparatus, energy",
    "Medical technology", "Semiconductors", "Measurement", "Optics",
    "Pharmaceuticals", "Other special machines", "Telecommunications"
]
def isTop(field):
    if(top_fields.__contains__(field)):
        return field
    else:
        return "Other"
    

# we will save all the field data for both the individuals and organizations so we can compare the field popularity in the end

indiv_data = patent_data.loc[patent_data["assignee_type"] == "Individual"]
org_data = patent_data.loc[patent_data["assignee_type"] == "Organization"]
indivField = indiv_data.loc[:].groupby("field").size().reset_index(name='patent_count')
orgField = org_data.loc[:].groupby("field").size().reset_index(name='patent_count')


patent_data["field"] = patent_data["field"].apply(isTop)

fields =  patent_data.loc[:].groupby("field").size().reset_index(name='patent_count')
fields = fields.sort_values(by="patent_count", ascending=False).reset_index(drop=True)
fields


,field,patent_count
0,Other,125796
1,Computer technology,51470
2,Digital communication,34940
3,Medical technology,23789
4,"Electrical machinery, apparatus, energy",22516
5,Measurement,15666
6,Semiconductors,14812
7,Optics,11255
8,Pharmaceuticals,8819
9,Other special machines,7593


In [125]:
# retrieve the data that we will use for the field comparisons
field_Data = patent_data.groupby("field")["years_to_grant"].agg({"mean","std"})
field_Data

,mean,std
field,,
Computer technology,2.474937,1.301975
Digital communication,2.449857,1.346251
"Electrical machinery, apparatus, energy",2.576834,1.351685
Measurement,2.792800,1.370461
Medical technology,3.197991,1.578045
Optics,2.396890,1.264479
Other,2.742822,1.493092
Other special machines,2.993810,1.527010
Pharmaceuticals,3.390294,1.838347


In [126]:
#compare the assignee_types
assignee_type = patent_data.loc[:].groupby("assignee_type").size().reset_index(name='patent_count')
assignee_type

,assignee_type,patent_count
0,Individual,2694
1,Organization,321128


In [127]:
# we balance the data by taking a random sample in the organizations table
balanced_org_data = org_data.sample(n = indiv_data.shape[0],random_state=42)
balanced_data = pd.concat([indiv_data,balanced_org_data])
balanced_assignee_type = balanced_data.loc[:].groupby("assignee_type").size().reset_index(name='patent_count')
balanced_assignee_type


,assignee_type,patent_count
0,Individual,2694
1,Organization,2694


In [128]:
#retrieve data for the organizations
balanced_assignee_type_data = balanced_data.groupby("assignee_type")["years_to_grant"].agg({"mean","std"})
balanced_assignee_type_data

,mean,std
assignee_type,,
Individual,2.706756,1.551724
Organization,2.688196,1.468077


Let's now move on to some plotting and analysis